In [1]:
import pandas as pd
from tabula.io import read_pdf
import sqlite3

In [2]:
df = read_pdf("../ecipdf1.pdf",pages='all') 

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [3]:
df= pd.concat(df,ignore_index=True)

In [4]:
df.rename(columns={
    'Sr No.': 'Sr_No',
    'Reference No  (URN)': 'Reference_No_URN',
    'Journal Date': 'Journal_Date',
    'Date of\rPurchase': 'Date_of_Purchase',
    'Date of Expiry': 'Date_of_Expiry',
    'Name of the Purchaser': 'Name_of_the_Purchaser',
    'Prefix': 'Prefix',
    'Bond\rNumber': 'Bond_Number',
    'Denominations': 'Denominations',
    'Issue Branch Code': 'Issue_Branch_Code',
    'Issue Teller': 'Issue_Teller',
    'Status': 'Status'
}, inplace=True)

In [5]:

df['Journal_Date'] = pd.to_datetime(df['Journal_Date'])
df['Date_of_Purchase'] = pd.to_datetime(df['Date_of_Purchase'], format='%d/%b/%Y')
df['Date_of_Expiry'] = pd.to_datetime(df['Date_of_Expiry'], format='%d/%b/%Y')

df['Denominations'] = df['Denominations'].str.replace(',', '').astype(int)
df['Journal_Date'] = df['Journal_Date'].dt.strftime('%Y-%m-%d')
df['Date_of_Purchase'] = df['Date_of_Purchase'].dt.strftime('%Y-%m-%d')
df['Date_of_Expiry'] = df['Date_of_Expiry'].dt.strftime('%Y-%m-%d')


conn = sqlite3.connect('data.db')

c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS bonds (
    Sr_No INTEGER,
    Reference_No_URN TEXT,
    Journal_Date TEXT,
    Date_of_Purchase TEXT,
    Date_of_Expiry TEXT,
    Name_of_the_Purchaser TEXT,
    Prefix TEXT,
    Bond_Number INTEGER,
    Denominations INTEGER,
    Issue_Branch_Code INTEGER,
    Issue_Teller INTEGER,
    Status TEXT
)
''')


df.to_sql('bonds', conn, if_exists='append', index=False)


conn.commit()
conn.close()

In [6]:
df = read_pdf("../ecipdf2.pdf",pages='all') 

In [7]:
df= pd.concat(df,ignore_index=True)

In [8]:
df.rename(columns={
    'Sr No.': 'Sr_No',
    'Date of\rEncashment': 'Date_of_Encashment',
    'Name of the Political Party': 'Name_of_the_Political_Party',
    'Account no. of\rPolitical Party': 'Account_no_of_Political_Party',
    'Prefix': 'Prefix',
    'Bond\rNumber': 'Bond_Number',
    'Denominations': 'Denominations',
    'Pay Branch\rCode': 'Pay_Branch_Code',
    'Pay Teller': 'Pay_Teller'
}, inplace=True)

In [9]:
df['Date_of_Encashment'] = pd.to_datetime(df['Date_of_Encashment'], format='%d/%b/%Y')
df['Denominations'] = df['Denominations'].str.replace(',', '').astype(int)
# Convert datetime columns to strings in the format suitable for SQL storage
df['Date_of_Encashment'] = df['Date_of_Encashment'].dt.strftime('%Y-%m-%d')

# Connect to SQLite database (or create it)
conn = sqlite3.connect('data.db')

# Create a cursor object
c = conn.cursor()

# Create table with appropriate data types
c.execute('''
CREATE TABLE IF NOT EXISTS political_bonds (
    Sr_No INTEGER,
    Date_of_Encashment TEXT,
    Name_of_the_Political_Party TEXT,
    Account_no_of_Political_Party TEXT,
    Prefix TEXT,
    Bond_Number INTEGER,
    Denominations INTEGER,
    Pay_Branch_Code TEXT,
    Pay_Teller INTEGER
)
''')

# Insert data into the table
df.to_sql('political_bonds', conn, if_exists='append', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [11]:
Query = '''SELECT SUM(Denominations) AS Total_Bond_Amount
FROM bonds
WHERE Name_of_the_Purchaser = 'CHOUDHARY GARMENTS'
AND Date_of_Purchase = '2019-04-12';
'''


In [15]:
conn = sqlite3.connect('data.db')

# Create a cursor object
cursor = conn.cursor()

In [24]:
sql_query = """
SELECT SUM(pb.Denominations) AS Total_Amount
FROM political_bonds pb
JOIN bonds b ON pb.Prefix = b.Prefix AND pb.Bond_Number = b.Bond_Number
WHERE pb.Name_of_the_Political_Party = 'AAM AADMI PARTY'
AND b.Name_of_the_Purchaser = 'DR. MANDEEP SHARMA'
AND strftime('%Y', pb.Date_of_Encashment) = '2022';
"""

try:
    # Execute the SQL query
    cursor.execute(sql_query)

    # Fetch the result
    total_bond_amount = cursor.fetchone()[0]

    # Print the result
    print( total_bond_amount)

except sqlite3.Error as e:
    print("Error executing SQLite query:", e)


1000000
